## Seasonal NAO skill comparison ##

In the decadal predictions, we observe a pretty significant drop in skill post-2005 initiailisation. As the dataset is limited to around 50 years and we have only observed one 'forecast bust' like this, we don't know whether Doug was just lucky or whether something is actually going wrong in the models in the recent period.

To explore this, we consider the seasonal hindcasts:

1. CSF-20C (Coupled Seasonal Forecast):This is a hindcast performed with the model fully coupled to the Nucleus for European Modelling of the Ocean (NEMO) ocean model, initialised from CERA-20C reanalysis data.
2. ASF-20C (Atmospheric Seasonal Forecasts): This is a hindcast performed with prescribed Sea Surface Temperature (SST) and sea-ice boundary conditions at the surface, initialised form ERA-20C reanalysis data.

These both run from initialisation in 1901-2009, so give a longer time period.

Strommen et al. (2023) propose that seasonal forecasts can be used to diagnose decadal forecast signals over a longer period. We plan to look at this in the context of 8 year running mean (years 2-9 of the decadal forecast) NAO predictability. In doing this, we want to explore the following questions:

* Was the recent drop in NAO skill an outlier in the longer period (1901-2020)?
* Have similar 'forecast bust' periods been observed before?
* Are there similar conditions occuring during previous bust periods?
    * i.e. divergence of model/obs SPNA SSTs

In [1]:
# Import relevant modules
import os
import sys

# Third party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Testing dask again
import dask
import dask.array as da

# Import cdo for regridding
from cdo import Cdo
cdo = Cdo()

In [2]:
# Import local dictionaries and functions
sys.path.append('/home/users/benhutch/skill-maps')

# Import dictionaries
import dictionaries as dic

# Set up the path where the functions are stored
sys.path.append('/home/users/benhutch/skill-maps/python')

# Import functions
import functions as func

# Import the NAO skill functions
import nao_skill_functions as nao_func

# Set up the path where the functions are stored
sys.path.append('/home/users/benhutch/skill-maps/rose-suite-matching')

# Import the suite functions
import nao_matching_seasons as nms_func

# Import the bootstrapping functions
import process_bs_values as pbs_func

### Data access issues ###

* ERA20C reanalysis not easily accessible?
* Who should I contact to try to get access?
* Maybe just look back to 1940 onwards (ERA5 window) first

In [3]:
# Find some of the ASF20C data on jasmin
# Set up the parameters
model = "ASF20C" # Alternative is "CSF20C"
variable = "SLP" # Sea level pressure
initialisation = "Nov" # Month of initialisation
year = 1901 # First year of the hindcast
member=1 # Ensemble member
region="Global" # For regridding

# Set up the base path
base_path_20c = "/badc/deposited2020/seasonal-forecasts-20thc/data"

# Find the specific file
folder_name = f"{variable}monthly_{model}_{initialisation}START_ENSmems"

# Set up the file name
file_name = f"{variable}monthly_{year}_M{member}.nc"

# Set up the full path
file_path = os.path.join(base_path_20c, model,
                            folder_name, file_name)

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file {file_path} exists")

# Set up directories in canari to store the regridded psl data
# Set up the base path
base_path = "/gws/nopw/j04/canari/users/benhutch"

# Form a new path
new_path_ASF20C = os.path.join(base_path, "seasonal",
                               "ASF20C", variable,
                                 f"{initialisation}_START")

# Same for the CSF20C
new_path_CSF20C = os.path.join(base_path, "seasonal",
                               "CSF20C", variable,
                                 f"{initialisation}_START")

# Check if the directories exist
if not os.path.exists(new_path_ASF20C):
    print(f"The directory {new_path_ASF20C} exists")
    # Create a new directory
    os.makedirs(new_path_ASF20C, exist_ok=False)

# Same for the CSF20C
if not os.path.exists(new_path_CSF20C):
    print(f"The directory {new_path_CSF20C} exists")
    # Create a new directory
    os.makedirs(new_path_CSF20C, exist_ok=False)

The file /badc/deposited2020/seasonal-forecasts-20thc/data/ASF20C/SLPmonthly_ASF20C_NovSTART_ENSmems/SLPmonthly_1901_M1.nc exists


In [4]:
# Find the gridspec for the file
gridspec_dir = "/home/users/benhutch/gridspec/"

# # Print all of the files within the directory
# print(os.listdir(gridspec_dir))

# Find gridspec-global.txt
gridspec_file = os.path.join(gridspec_dir, "gridspec-global.txt")

# Print the contents of the file
print(open(gridspec_file).read())

gridtype=lonlat
xfirst=-180
xinc=2.5
xsize=144
yfirst=-90
yinc=2.5
ysize=72


In [5]:
# Find the directory where all of the data to be regridded is stored
dir = os.path.join(base_path_20c, model,
                            folder_name)

# # Print all of the files within the directory
# print(os.listdir(dir))

# # Find the files within the directory
# files = os.listdir(dir)

# # Limit files to the first 10
# files = files[:10]

# # Loop through the files
# for file in files:
#     # Set up the output file name
#     output_file = os.path.join(new_path_ASF20C, f"{file[:-3]}_rg.nc")

#     # If the output file already exists, skip
#     if os.path.exists(output_file):
#         print(f"The file {output_file} exists")
#         continue

#     # Set up the input file name
#     input_file = os.path.join(dir, file)

#     # perform the regridding
#     cdo.remapbil(gridspec_file, input=input_file,
#                  output=output_file)

### Time ###

The timesteps for the seasonal forecasts are set up in a weird way. We are going to assume that if the forecast is initialized on the 1st November and then run forwards for four months, then the first timestep is the November mean, the second December, then January and February. Therefore, to get the DJF mean, we will take the average of all time steps but the first.

In [6]:
#Import pandas
import pandas as pd

# # Find my directory with the regridded data
# # Print the files contained within new_path_ASF20C
# print(os.listdir(new_path_ASF20C))

# Create an array of initialisation years
years = np.arange(1901, 2011)

# For a single file, collapse this into a DJF mean
# First calculate the anomaly field
# Then calculate the DJF mean anomalies
# Then azores gridbox mean - iceland gridbox mean
# Do this for a single file first as a test
test_file = os.path.join(new_path_ASF20C, "SLPmonthly_1901_M1_rg.nc")

# Read in the file
ds = xr.open_dataset(test_file)

ds

# # Extract the lats and lons
lats = ds['lat']
lons = ds['lon']

# Generate an array of years from 1901 to 2009
years = np.arange(1901, 2010)

# Generate an array for the members from 1 to 50
members = np.arange(1, 51)

# Print the shape of the data
print(np.shape(lats))
print(np.shape(lons))
print(np.shape(years))
print(np.shape(members))

# Extract the data for the variable: 'MSL_GDS0_SFC'
data = ds['MSL_GDS0_SFC']

# Print the shape of the data
print(data.shape)

# Take the mean over the time dimension
djf_mean = data[1:, :, :].mean(axis=0)

# Print the shape of the data
print(djf_mean.shape)

# Print the data
print(djf_mean)

(72,)
(144,)
(109,)
(50,)
(4, 72, 144)
(72, 144)
<xarray.DataArray 'MSL_GDS0_SFC' (lat: 72, lon: 144)>
array([[100824.6 , 100824.6 , 100824.6 , ..., 100824.6 , 100824.6 ,
        100824.6 ],
       [100243.15, 100247.77, 100253.4 , ..., 100259.94, 100253.44,
        100247.77],
       [ 99846.35,  99874.48,  99851.52, ..., 100026.48,  99937.15,
         99880.23],
       ...,
       [102820.85, 102822.23, 102824.94, ..., 102804.94, 102813.23,
        102817.81],
       [102873.69, 102879.85, 102886.85, ..., 102854.85, 102861.85,
        102869.35],
       [102807.19, 102810.77, 102814.1 , ..., 102792.31, 102797.56,
        102802.48]], dtype=float32)
Coordinates:
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5


In [ ]:
# Start the dask client
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

In [7]:
# Create an empty Dask array to store the data
asf_psl_field = da.empty([len(years), len(members), len(lats), len(lons)])

# Print the shape of the array
print(asf_psl_field.shape)

# Define a function to process a single year and member
def process_year_member(year, member):
    # Print the year and member
    print("Processing year: ", year, " member: ", member)

    # Set up the file name
    file_name = f"SLPmonthly_{year}_M{member}_rg.nc"

    # Set up the file path
    file_path = os.path.join(new_path_ASF20C, file_name)

    # Read in the file
    ds = xr.open_dataset(file_path, chunks={})

    # Extract the data for the variable: 'MSL_GDS0_SFC'
    data = ds['MSL_GDS0_SFC']

    # Take the mean over the time dimension
    djf_mean = data[1:, :, :].mean(axis=0)

    # Append the data to the Dask array
    asf_psl_field[year-1901, member-1, :, :] = djf_mean

    # Return the Dask array
    return asf_psl_field

# Loop through the years and members
for year in years:
    print("Processing year: ", year)
    for member in members:
        print("Processing member: ", member)

        # Use das.delayed to delay the function
        results = dask.delayed(process_year_member)(year, member)

# Start the Dask computation
asf_psl_field = dask.compute(*results)

# Print the array
print(asf_psl_field.shape)

(109, 50, 72, 144)


Processing year:  1928  member:  34
Processing year:  1949  member:  35
Processing year:  1993  member:  17
Processing year:  1987  member:  35
Processing year:  1987  member:  27
Processing year:  1970  member:  17
Processing year:  2003  member:  18
Processing year:  1948  member:  23
Processing year:  1937  member:  20
Processing year:  2004  member:  26
Processing year:  2007  member:  18
Processing year:  1953  member:  9
Processing year:  1922  member:  1
Processing year:  2004  member:  45
Processing year:  1970  member:  10
Processing year:  1994  member:  20
Processing year:  2008  member:  32
Processing year:  1971  member:  47
Processing year:  1980  member:  40
Processing year:  1971  member:  8
Processing year:  1946  member:  18
Processing year:  1913  member:  14
Processing year:  1968  member:  13
Processing year:  1921  member:  1
Processing year:  1910  member:  15
Processing year:  1975  member:  21
Processing year:  2006  member:  13
Processing year:  1992  member: 

AttributeError: 'tuple' object has no attribute 'shape'